In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from operator import itemgetter
from datetime import date, datetime

In [3]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

## data EDA
- 데이터 : kodex200.csv
- 추출일자 : 2002.10.14 ~ 2022.08.25

In [4]:
def data_preprocessing(stock):

    stock.columns = ["date","open","high","low","close","volume","multiple_volume","total_volume"]
    stock = stock.sort_values("date").reset_index(drop=True)

    datetime_list = [datetime.strptime(i,"%Y-%m-%d").date() for i in stock.date]
    stock["datetime"] = datetime_list
    
    #2 per
    per_list = [0,]
    for i in range(1,len(stock)):
        plus_per = ((stock.close[i]/stock.close[i-1])-1) * 100
        per_list.append(plus_per)
    stock["per"] = per_list

    
    #3 multiple_volume
    stock["multiple_volume"] = stock["close"] * stock["volume"]
    
    return stock

## 변동성-거래량  score

### 변동성 점수
- per을 통한 변동성 점수 계산
- per의 지수가중이동편차

In [5]:
def calcul_volatility(stock, stock_FG, anti_join):
    
    lam = 0.94
    mu = stock.per[0]
    sigma = 0

    sigma_list = []
    for i in range(len(stock.close)):
        sigma = (lam * (sigma)) + ((1-lam) * (stock.per[i]**2))
        sigma_list.append(sigma)

    sqrt_sigma = np.sqrt(sigma_list)
    log_sigma = np.log(sqrt_sigma)

    stock["log_sigma"] = log_sigma

    Volatility_list = []
    
    
    for i in anti_join.date:
        # print(i)
        process_idx = stock[stock.date==i].index[0]
        tmp_data = stock[(process_idx-365):process_idx]

        # data condition
        if len(tmp_data) == 365 :

            sigma365 = tmp_data['log_sigma']
            sigma365[sigma365==-np.inf] = np.nan

            mu_setting = np.nanmean(sigma365)
            std_setting = np.nanstd(sigma365)

            set1 = np.max([-4, ((sigma365.values[364] -  mu_setting)/std_setting) ])
            set2 = np.min([set1,4])
            Volatility_list.append(set2)
            
    return Volatility_list


### 거래량 점수
- 원화환산 거래량 값을 사용해 거래량 점수를 계산한다
- 단기 거래량과 장기 거래량의 값을 사용한다
- lambda 를 통해서 지수가중이동평균의 가중치를 변경한다
    - 첫 N일의 정보포함률이 63.2%로 수렴하는 수치

In [6]:
def calcul_volume(stock, stock_FG, anti_join):

    lambda_20 = 1-(1/20)
    lambda_60 = 1-(1/60)

    volume_sigma20_list = []
    volume_sigma60_list = []
    Volume_list = []
    
    for i in anti_join.date:
        process_idx = stock[stock.date==i].index[0]    
        # list set
        list1 = []
        list2 = []
        for j in range(process_idx+1):
            volume_sigma_20 = (1-lambda_20) * (lambda_20**j) * stock.multiple_volume[process_idx-j]
            volume_sigma_60 = (1-lambda_60) * (lambda_60**j) * stock.multiple_volume[process_idx-j]
            list1.append(volume_sigma_20)
            list2.append(volume_sigma_60)

        V20 = np.sum(list1)
        V60 = np.sum(list2)
        V_20 = np.log(stock.iloc[process_idx].multiple_volume/V20)
        V_60 = np.log(stock.iloc[process_idx].multiple_volume/V60)
        
        set1 = np.max([-4, ((V_20+V_60)/2)])
        set2 = np.min([set1, 4])

        Volume_list.append(set2)
        
    return Volume_list

### 변동성-거래량 점수

- 위 단계에서 계산한 (1)Voltaility 와 (2)Volume 스코어의 값을 평균내어 변동성 거래량 점수를 계산한다 
- 점수의 스케일은 0~1까지의 스케일이다.

In [7]:
def volatility_volume_score(Volatility_list, Volume_list):
    S1_list = []
    for i in range(len(Volatility_list)):
        set1 = np.max([-4,float(Volatility_list[i] + Volume_list[i])])
        set2 = np.min([ 4, set1])
        set3 = set2/8 + 0.5
        S1_list.append(set3)
    return S1_list


## 모멘텀 score

- kodex 종가의 가중이동평균을 계산한후 실제가와 이동평균값의 이격도를 통해 모멘텀 점수를 계산한다
- 변동성-거래량 점수에 의해 장기이평선 단기이평선 가중을 변경한다 
- 시장의 변동성과 거래량이 높은 (거래량 급등/급락)하는 시장에서 투자자들은 단기간 추세에 민감하게 반응
- 변동성 거래량 점수가 높다면 단기 가중치를 더욱 크게 한다

In [9]:
def momentum_score(S1_list, stock, stock_FG, anti_join):
    S1_score = list(stock_FG.S1_score)
    S1_score.extend(S1_list)

    short_alpha = [(i*9)+1 for i in S1_score]
    long_alpha = [10 - ((i*9)+1) for i in S1_score]

    lambda_30 = 1-(1/30)
    lambda_7 = 1-(1/7)

    close_mu30_list = []
    close_mu7_list = []
    
    S2_list = []
    for i in anti_join.date:
        # list set
        list1 = []
        list2 = []

        process_idx = stock[stock.date==i].index[0]
        for j in range(process_idx+1):
            close_mu_30 = (1-lambda_30) * (lambda_30**j) * stock.close[process_idx-j]
            close_mu_7 = (1-lambda_7) * (lambda_7**j) * stock.close[process_idx-j]
            list1.append(close_mu_30)
            list2.append(close_mu_7)

        V30 = np.sum(list1)
        V7 = np.sum(list2)

        X_short = (stock.iloc[process_idx].close - V7) / V7
        X_long = (stock.iloc[process_idx].close - V30) / V30

        products_short = short_alpha[process_idx-365] * X_short
        products_long = long_alpha[process_idx-365] * X_long
        product_momentum = products_short + products_long
        
        C = 16.387308
        set1 = (product_momentum * C) / 10
        S2_list.append(set1)
        
    return S2_list

    


## FG score

- S1_list : 변동성 거래량 점수
    - 스케일 : 0 ~ 1 
    
- S2_list : 모멘텀 점수
    - 스케일 : 실수 범위

- S1_list : FG score
    - 공포탐욕지수
    
- 케이스별 시나리오
    - case1 : 변동성이 큰 상태에서 모멘텀도 크다
        - 탐욕지수가 매우 커진다
    - case2 : 변동성이 작은 상태에서 모멘텀도 크다
        - 탐욕지수가 비교적 상승하고자 한다
        - 지수에 영향이 별로 없다 (0.5에 가깝게 나타난다) 
    - case3 : 변동성이 큰 상태에서 모멘텀이 작다
        - 탐욕지수가 매우 작아진다
    - case4 : 변동성이 작은상태에서 모멘텀이 크다
        - 탐욕지수가 비교적 하락하고자 한다
        - 지수에 영향이 별로 없다 (0.5에 가깝게 나타난다)
        
- 비고
    - 변동성 거래량 점수
        - 0~1이기 때문에 중립에서 양분되어 공포와 탐욕이 커질수록 커지는 값이다 
    - 모멘텀 점수
        - 실수의 범위에서 이격도를 뜻한다

- (1)변동성-거래량 점수와 (2)모멘텀 점수를 곱한다
- 곱한 값을 로지스틱 함수에 대입하여 최종 공포탐욕지수 스코어를 산출한다

In [10]:
def calcul_FGscore(S1_list, S2_list):
    
    FG_list = []
    
    for i in range(len(S1_list)):
        S_value = -(S1_list[i] * S2_list[i])
        FG_score = 1/(1+np.exp(S_value))
        FG_list.append(FG_score)
        
    return FG_list


In [11]:
def FG_index(stock_df, stock_FG):
    try:
        print("preprocessing")
        stock = data_preprocessing(stock_df)
        datetime_list = stock['datetime']

        process_stock = stock[365:]
        outer = process_stock.merge(stock_FG, how = "outer", on = "date", indicator = True)
        anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

        Volatility_list = calcul_volatility(stock, stock_FG, anti_join)

        Volume_list = calcul_volume(stock, stock_FG, anti_join)

        S1_list = volatility_volume_score(Volatility_list, Volume_list)

        S2_list = momentum_score(S1_list, stock, stock_FG, anti_join)

        print("FG score")
        FG_score = calcul_FGscore(S1_list, S2_list)

        # data processing
        tmp_stock = stock[(len(stock)-len(anti_join)):(len(stock))]
        tmp_stock = tmp_stock[['date','close','volume','multiple_volume','per']]
        tmp_stock['Volatility_list'] = Volatility_list
        tmp_stock['Volume_list'] = Volume_list
        tmp_stock['S1_score'] = S1_list
        tmp_stock['S2_score'] = S2_list
        tmp_stock['FG_score'] = FG_score
        final_data = pd.concat([stock_FG, tmp_stock]).reset_index(drop=True)

        return final_data
    except Exception as e:    
        print('예외가 발생했습니다.', e)
        return []



In [12]:
# final_data = FG_index(kodex)

## Plotting

In [19]:
# px.line(final_data, x = 'date', y='FG_score',
#         hover_data={"date": "|%B %d, %Y"})

In [20]:
# fig = make_subplots(specs = [[{'secondary_y':True}]])
# fig.add_trace(
#     go.Scatter(x=final_data.date, y = final_data.FG_score, name = "FG_score"
#                , hovertemplate = 'date = %{x}<br>'+
#                                  'value = %{y}'),
#     secondary_y = False,
# )
# fig.add_trace(
#     go.Scatter(x=final_data.date, y = final_data.close, name = "kodex200"
#                , hovertemplate = 'date = %{x}<br>'+
#                                  'value = %{y}'),
#     secondary_y = True,
# )
# fig.add_trace(
#     go.Scatter(x=final_data.date, y = [0.5]*4550, name = 'middle'),
#     secondary_y = False,
# )

# fig.update_layout(
#     title_text = "FG score", hovermode="x")
# fig.update_xaxes(title_text = "date", rangeslider_visible = True)
# fig.update_yaxes(title_text = "FG_score", secondary_y = False)
# fig.update_yaxes(title_text = "stock_close", secondary_y = True)
# # specific line

# fig.show()